In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import torch
from modeling_llama import LlamaForCausalLM
from transformers import AutoTokenizer
from decoding import clip_input, infer_input_ids
from rouge_score import rouge_scorer
import numpy as np

/mnt/users/w32zhong/anaconda/lg/lib/python3.8/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/mnt/users/w32zhong/anaconda/lg/lib/python3.8/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


(Re-)Loading modeling...


In [2]:
torch.nn.Linear.reset_parameters = lambda x: None
model = LlamaForCausalLM.from_pretrained('NousResearch/Llama-2-13b-chat-hf', torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained('NousResearch/Llama-2-13b-chat-hf')
device='cuda:2'
model = model.to(device).eval()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/mnt/users/w32zhong/anaconda/lg/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/mnt/users/w32zhong/anaconda/lg/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [3]:
# -*- encoding:utf-8 -*-
import transformers
print(torch.version.cuda)
print(torch.__version__) 
print(transformers.__version__)

11.8
2.3.0.dev20231230+cu118
4.33.1


In [4]:
_attn_skip_layer_id_set, _mlp_skip_layer_id_set =model.get_skip_layers()
print(_attn_skip_layer_id_set, _mlp_skip_layer_id_set)

[3, 5, 6, 8, 10, 11, 14, 15, 18, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37] [6, 9, 10, 11, 15, 24, 25, 27, 28, 35]


In [5]:
seed=42
torch.manual_seed(seed)
np.random.seed(seed)

In [8]:
from datasets import load_dataset
n_shot = 1
task_name = 'cnndm'
prompt_shots = ''
if task_name == 'xsum':
    data = load_dataset('xsum', split='test').shuffle(seed=seed).select(range(1000))
    shots = load_dataset('xsum',split='train').shuffle(seed=seed).select(range(n_shot))
    prompt_keys=['document','summary']
elif task_name == 'cnndm':
    data = load_dataset('cnn_dailymail', name='3.0.0', split='test') .shuffle(seed=seed).select(range(1000))
    shots = load_dataset('cnn_dailymail', name='3.0.0', split='train').shuffle(seed=seed).select(range(n_shot))
    prompt_keys=['article','highlights']
for i in range(n_shot):
    prompt = 'Article: ' + shots[i][prompt_keys[0]] + '\nSummary: ' + shots[i][prompt_keys[1]].replace('\n', '') + '\n'
    prompt_shots += prompt

In [9]:
print(prompt_shots)

Article: By . Anthony Bond . PUBLISHED: . 07:03 EST, 2 March 2013 . | . UPDATED: . 08:07 EST, 2 March 2013 . Three members of the same family who died in a static caravan from carbon monoxide poisoning would have been unconscious 'within minutes', investigators said today. The bodies of married couple John and Audrey Cook were discovered alongside their daughter, Maureen, at the mobile home they shared on Tremarle Home Park in Camborne, west Cornwall. The inquests have now opened into the deaths last Saturday, with investigators saying the three died along with the family's pet dog, of carbon monoxide poisoning from a cooker. Tragic: The inquests have opened into the deaths of three members of the same family who were found in their static caravan last weekend. John and Audrey Cook are pictured . Awful: The family died following carbon monoxide poisoning at this caravan at the Tremarle Home Park in Camborne, Cornwall . It is also believed there was no working carbon monoxide detector i

In [15]:
import sys
rouge=rouge_scorer.RougeScorer(['rouge2'], use_stemmer=True)
main_metrics = {'rouge2_base':[], 
                'rouge2_essg1':[], 'rouge2_essg2':[], 'rouge2_essg3':[], 'rouge2_essg4':[], 
                'rouge2_essg_autoth':[],
                'time_base':[], 
                'time_essg1':[], 'time_essg2':[], 'time_essg3':[], 'time_essg4':[], 
                'time_essg_autoth':[],
                'token_time_base':[], 
                'token_time_essg1':[], 'token_time_essg2':[], 'token_time_essg3':[], 'token_time_essg4':[], 
                'token_time_essg_autoth':[],
                'matchness_essg1':[],'num_drafted_tokens_essg1':[],
                'matchness_essg2':[],'num_drafted_tokens_essg2':[],
                'matchness_essg3':[],'num_drafted_tokens_essg3':[],
                'matchness_essg4':[],'num_drafted_tokens_essg4':[],
                'matchness_essg_autoth':[],'num_drafted_tokens_essg_autoth':[]}

if True:
    for i,x in enumerate(data):
        print(x, end='\n\n')
        input_ids = clip_input(tokenizer, x, task_name, max_new_tokens=512,prompt_shots=prompt_shots)
        
        if i == 0:
            th_stop_draft_essg1 = 0.20
            th_stop_draft_essg2 = 0.40
            th_stop_draft_essg3 = 0.60
            th_stop_draft_essg4 = 0.80
            th_stop_draft_essg_autoth  = 0.60
        else:
            th_stop_draft_essg1 = result_essg1['th_stop_draft']
            th_stop_draft_essg2 = result_essg2['th_stop_draft']
            th_stop_draft_essg3 = result_essg3['th_stop_draft']
            th_stop_draft_essg4 = result_essg4['th_stop_draft']
            th_stop_draft_essg_autoth = result_essg_autoth['th_stop_draft']
        print('essg th1: {:.4f}, essg th2: {:.4f}, essg th3: {:.4f}, essg th4: {:.4f}, essg autoth: {:.4f} \n'.format(
        th_stop_draft_essg1, th_stop_draft_essg2, th_stop_draft_essg3, th_stop_draft_essg4, th_stop_draft_essg_autoth))
        result_base = infer_input_ids(model, tokenizer, input_ids, generate_fn='base',
                    max_new_tokens=512, do_sample=False, early_stop=True)
        print('result_base done')
        result_essg1 = infer_input_ids(model, tokenizer, input_ids, generate_fn='essg', 
                    max_new_tokens=512, early_stop=True, max_step_draft=12, 
                    th_stop_draft=th_stop_draft_essg1, auto_th_stop_draft=False,
                    do_sample=False, do_sample_draft=False)
        print('result_essg1 done')
        result_essg2 = infer_input_ids(model, tokenizer, input_ids, generate_fn='essg', 
                    max_new_tokens=512, early_stop=True, max_step_draft=12, 
                    th_stop_draft=th_stop_draft_essg2, auto_th_stop_draft=False,
                    do_sample=False, do_sample_draft=False)
        print('result_essg2 done')
        result_essg3 = infer_input_ids(model, tokenizer, input_ids, generate_fn='essg', 
                    max_new_tokens=512, early_stop=True, max_step_draft=12, 
                    th_stop_draft=th_stop_draft_essg3,  auto_th_stop_draft=False,
                    do_sample=False, do_sample_draft=False)
        print('result_essg3 done')
        result_essg4 = infer_input_ids(model, tokenizer, input_ids, generate_fn='essg', 
                    max_new_tokens=512, early_stop=True, max_step_draft=12, 
                    th_stop_draft=th_stop_draft_essg4,  auto_th_stop_draft=False,
                    do_sample=False, do_sample_draft=False)
        print('result_essg4 done')
        result_essg_autoth = infer_input_ids(model, tokenizer, input_ids, generate_fn='essg', 
                    max_new_tokens=512, early_stop=True, max_step_draft=12, 
                    th_stop_draft=th_stop_draft_essg_autoth, auto_th_stop_draft=True, auto_parameters=[1,0.50,0.90,1e-2,0.90],
                    do_sample=False, do_sample_draft=False)
        print('result_essg_autoth done')

        if len(result_base['completion']) < 5 or ('.....' in result_base['completion'][:5]):
            print("too short, skip")
            continue
        
        if task_name == 'xsum':
            references = x['summary']
        elif task_name =='cnndm':
            references = x['highlights']
            
        results = [
            ('base', result_base),
            ('essg1', result_essg1),
            ('essg2', result_essg2),
            ('essg3', result_essg3),
            ('essg4', result_essg4),
            ('essg_autoth', result_essg_autoth)
        ]

        for key, result in results:
            main_metrics['time_' + key].append(result['time'])
            main_metrics['token_time_' + key].append(result['time'] / result['generate_ids'].shape[1])
            if key != 'base':
                main_metrics['matchness_' + key].append(result['matchness'])
                main_metrics['num_drafted_tokens_' + key].append(result['num_drafted_tokens'])
            clip_pred = result['completion'].find("\nArticle:")
            if clip_pred > 0:
                prediction = result['completion'][:clip_pred]
            else:
                prediction = result['completion']
            rouge_score = rouge.score(prediction, references)
            main_metrics['rouge2_' + key].append(rouge_score['rouge2'].fmeasure)

        metric = {
            'mean rouge-2 base':np.mean(main_metrics['rouge2_base']),
            f'mean rouge-2 essg th {th_stop_draft_essg1}':np.mean(main_metrics['rouge2_essg1']),
            f'mean rouge-2 essg th {th_stop_draft_essg2}':np.mean(main_metrics['rouge2_essg2']),
            f'mean rouge-2 essg th {th_stop_draft_essg3}':np.mean(main_metrics['rouge2_essg3']),
            f'mean rouge-2 essg th {th_stop_draft_essg4}':np.mean(main_metrics['rouge2_essg4']),
            'mean rouge-2 essg autoth':np.mean(main_metrics['rouge2_essg_autoth']),
            'mean time base':np.mean(main_metrics['time_base']),
            f'mean time essg th {th_stop_draft_essg1}':np.mean(main_metrics['time_essg1']),
            f'mean time essg th {th_stop_draft_essg2}':np.mean(main_metrics['time_essg2']),
            f'mean time essg th {th_stop_draft_essg3}':np.mean(main_metrics['time_essg3']),
            f'mean time essg th {th_stop_draft_essg4}':np.mean(main_metrics['time_essg4']),
            'mean time essg autoth':np.mean(main_metrics['time_essg_autoth']),
            f'E2E mean speed up essg th {th_stop_draft_essg1}':np.mean(main_metrics['time_base'])/np.mean(main_metrics['time_essg1']),
            f'E2E mean speed up essg th {th_stop_draft_essg2}':np.mean(main_metrics['time_base'])/np.mean(main_metrics['time_essg2']),
            f'E2E mean speed up essg th {th_stop_draft_essg3}':np.mean(main_metrics['time_base'])/np.mean(main_metrics['time_essg3']),
            f'E2E mean speed up essg th {th_stop_draft_essg4}':np.mean(main_metrics['time_base'])/np.mean(main_metrics['time_essg4']),
            'E2E mean speed up essg autoth':np.mean(main_metrics['time_base'])/np.mean(main_metrics['time_essg_autoth']),
            'mean token time base':np.mean(main_metrics['token_time_base']),
            f'mean token time essg th {th_stop_draft_essg1}':np.mean(main_metrics['token_time_essg1']),
            f'mean token time essg th {th_stop_draft_essg2}':np.mean(main_metrics['token_time_essg2']),
            f'mean token time essg th {th_stop_draft_essg3}':np.mean(main_metrics['token_time_essg3']),
            f'mean token time essg th {th_stop_draft_essg4}':np.mean(main_metrics['token_time_essg4']),
            'mean token time essg autoth':np.mean(main_metrics['token_time_essg_autoth']),  
            f'E2E mean token speed up essg th {th_stop_draft_essg1}':np.mean(main_metrics['token_time_base'])/np.mean(main_metrics['token_time_essg1']),
            f'E2E mean token speed up essg th {th_stop_draft_essg2}':np.mean(main_metrics['token_time_base'])/np.mean(main_metrics['token_time_essg2']),
            f'E2E mean token speed up essg th {th_stop_draft_essg3}':np.mean(main_metrics['token_time_base'])/np.mean(main_metrics['token_time_essg3']),
            f'E2E mean token speed up essg th {th_stop_draft_essg4}':np.mean(main_metrics['token_time_base'])/np.mean(main_metrics['token_time_essg4']),
            'E2E mean token speed up essg autoth':np.mean(main_metrics['token_time_base'])/np.mean(main_metrics['token_time_essg_autoth']),          
            f'mean matchness essg th {th_stop_draft_essg1}':np.mean(main_metrics['matchness_essg1']),
            f'mean matchness essg th {th_stop_draft_essg2}':np.mean(main_metrics['matchness_essg2']),
            f'mean matchness essg th {th_stop_draft_essg3}':np.mean(main_metrics['matchness_essg3']),
            f'mean matchness essg th {th_stop_draft_essg4}':np.mean(main_metrics['matchness_essg4']),
            'mean matchness essg autoth':np.mean(main_metrics['matchness_essg_autoth']),
            f'mean num_drafted_tokens essg th {th_stop_draft_essg1}':np.mean(main_metrics['num_drafted_tokens_essg1']),
            f'mean num_drafted_tokens essg th {th_stop_draft_essg2}':np.mean(main_metrics['num_drafted_tokens_essg2']),
            f'mean num_drafted_tokens essg th {th_stop_draft_essg3}':np.mean(main_metrics['num_drafted_tokens_essg3']),
            f'mean num_drafted_tokens essg th {th_stop_draft_essg4}':np.mean(main_metrics['num_drafted_tokens_essg4']),
            'mean num_drafted_tokens essg autoth':np.mean(main_metrics['num_drafted_tokens_essg_autoth']),
        }
        for key, value in metric.items():
            if isinstance(value, float):
                metric[key] = f"{value:.4f}"

        # print(f'data {i},{metric}')
        print(f'data {i},{metric} \n')
        break

{'article': '(CNN) I see signs of a revolution everywhere. I see it in the op-ed pages of the newspapers, and on the state ballots in nearly half the country. I see it in politicians who once preferred to play it safe with this explosive issue but are now willing to stake their political futures on it. I see the revolution in the eyes of sterling scientists, previously reluctant to dip a toe into this heavily stigmatized world, who are diving in head first. I see it in the new surgeon general who cites data showing just how helpful it can be. I see a revolution in the attitudes of everyday Americans. For the first time a majority, 53%, favor its legalization, with 77% supporting it for medical purposes. Support for legalization has risen 11 points in the past few years alone. In 1969, the first time Pew asked the question about legalization, only 12% of the nation was in favor. I see a revolution that is burning white hot among young people, but also shows up among the parents and gran

'E2E mean token speed up essg th 0.2': '0.6583', 
'E2E mean token speed up essg th 0.4': '0.4406',
'E2E mean token speed up essg th 0.6': '1.0358',
'E2E mean token speed up essg th 0.8': '0.7673',
'E2E mean token speed up essg autoth': '1.0797'.